# Buildings that have ongoing processes

Many buildings have ongoing process that happen over time. This is controlled by an AZ timer function.
```python
@app.function_name(name="factionBuildingTimer")
@app.schedule(schedule="0 */10 * * * *", 
              arg_name="mytimer",
              run_on_startup=RUNNING_LOCALLY)
```



In [1]:
import numpy as np
import pandas as pd
import logging
import sys, os, yaml, ssl, asyncio

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# mapping to the modules that make the app
sys.path.insert(0, "../..")


In [6]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

from app.connectors import cmdb_graph
from app.objects import structures
c = cmdb_graph.CosmosdbClient()

Starting with getting a list of the structures and listing them. 

In [7]:
faction_res = structures.get_faction_pop_structures(c)
pd.DataFrame(faction_res)

2024-10-13 11:19:04,550 - DEBUG - Using selector: SelectSelector
2024-10-13 11:19:04,998 - INFO - EXOADMIN: number of items: 1


,faction,pop,structure,action
0,"{'wealth': -457, 'name': 'Dalram', 'objid': '6...","{'isIdle': 'true', 'name': 'Dalram Leymon', 'o...","{'name': 'Department of Forestry', 'objid': '6...",structure


Here we are attaching `structure` to the message so that the EventHub trigger can catch it and process it. Other than that, it does nothing. 

In [8]:
message = faction_res[0]
message['action']

'structure'

In [9]:
message.keys()

dict_keys(['faction', 'pop', 'structure', 'action'])

In [10]:
message['structure']

{'name': 'Department of Forestry',
 'objid': '6058595786349',
 'ownedBy': '2775406017619',
 'type': 'forestry',
 'description': 'Generates organic foodstuffs',
 'populations will consume food before consuming natural resources': 'None',
 'planet_requirements': '{isHabitable: True}',
 'faction_augments': '{wealth: -3}',
 'renews_location_resource': '{organics: 10}',
 'owned_by': 'pop',
 'effort': '10',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'building',
 'id': '6058595786349'}

In [11]:
message['pop']

{'isIdle': 'true',
 'name': 'Dalram Leymon',
 'objid': '2775406017619',
 'conformity': 0.802,
 'literacy': 0.798,
 'aggression': 0.315,
 'constitution': 0.19,
 'health': 0.7,
 'isIn': '6953342521081',
 'industry': 0.2525,
 'wealth': 0.5252,
 'factionLoyalty': 0.812,
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'pop',
 'id': '2775406017619'}

In [12]:
message['faction']

{'wealth': -457,
 'name': 'Dalram',
 'objid': '6953342521081',
 'lat': 0,
 'long': 0,
 'infrastructure': 0,
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'faction',
 'id': '6953342521081'}

## faction_augments

In [13]:
structures.augemt_faction(c, message)

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\willi\miniconda3\envs\exoplanets\lib\logging\__init__.py", line 1085, in emit
    msg = self.format(record)
  File "c:\Users\willi\miniconda3\envs\exoplanets\lib\logging\__init__.py", line 929, in format
    return fmt.format(record)
  File "c:\Users\willi\miniconda3\envs\exoplanets\lib\logging\__init__.py", line 668, in format
    record.message = record.getMessage()
  File "c:\Users\willi\miniconda3\envs\exoplanets\lib\logging\__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "c:\Users\willi\miniconda3\envs\exoplanets\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\willi\miniconda3\envs\exoplanets\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\willi\miniconda3\envs\exoplanets\lib\site-packages\ipykernel_launcher.py", l

{'wealth': -3}

In [14]:
fac_id = message['faction']['objid']
check_faction = f"""
        g.V().has('label','faction').has('objid','{fac_id}').valueMap()
    """
c.run_query(check_faction)
c.clean_nodes(c.res)

2024-10-13 11:19:10,061 - DEBUG - Using selector: SelectSelector


[{'wealth': -460,
  'name': 'Dalram',
  'objid': '6953342521081',
  'lat': 0,
  'long': 0,
  'infrastructure': 0,
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'faction',
  'id': '6953342521081'}]

# The ongoing faction process in Azure Functions


In [15]:
messages = structures.get_faction_pop_structures(c)
pd.DataFrame(messages)

2024-10-13 11:19:11,462 - DEBUG - Using selector: SelectSelector
2024-10-13 11:19:11,809 - INFO - EXOADMIN: number of items: 1


,faction,pop,structure,action
0,"{'wealth': -460, 'name': 'Dalram', 'objid': '6...","{'isIdle': 'true', 'name': 'Dalram Leymon', 'o...","{'name': 'Department of Forestry', 'objid': '6...",structure


In [16]:
message = messages[0]
message.keys()

dict_keys(['faction', 'pop', 'structure', 'action'])

In [17]:
message['structure']  

{'name': 'Department of Forestry',
 'objid': '6058595786349',
 'ownedBy': '2775406017619',
 'type': 'forestry',
 'description': 'Generates organic foodstuffs',
 'populations will consume food before consuming natural resources': 'None',
 'planet_requirements': '{isHabitable: True}',
 'faction_augments': '{wealth: -3}',
 'renews_location_resource': '{organics: 10}',
 'owned_by': 'pop',
 'effort': '10',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'building',
 'id': '6058595786349'}

In [18]:
print(f"EXOADMIN: process_structure, structure: {message['structure']['name']}: {message['structure']['objid']}")

EXOADMIN: process_structure, structure: Department of Forestry: 6058595786349


## Augmenting the resources of a location

In [20]:
message

{'faction': {'wealth': -460,
  'name': 'Dalram',
  'objid': '6953342521081',
  'lat': 0,
  'long': 0,
  'infrastructure': 0,
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'faction',
  'id': '6953342521081'},
 'pop': {'isIdle': 'true',
  'name': 'Dalram Leymon',
  'objid': '2775406017619',
  'conformity': 0.802,
  'literacy': 0.798,
  'aggression': 0.315,
  'constitution': 0.19,
  'health': 0.7,
  'isIn': '6953342521081',
  'industry': 0.2525,
  'wealth': 0.5252,
  'factionLoyalty': 0.812,
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'pop',
  'id': '2775406017619'},
 'structure': {'name': 'Department of Forestry',
  'objid': '6058595786349',
  'ownedBy': '2775406017619',
  'type': 'forestry',
  'description': 'Generates organic foodstuffs',
  'populations will consume food before consuming natural resources': 'None',
  'planet_requirements': '{isHabitable: True}',
  'faction_augments': '{wealth: -3}',
  'renews_location_resource': '{organics: 1

In [21]:
message['structure']

{'name': 'Department of Forestry',
 'objid': '6058595786349',
 'ownedBy': '2775406017619',
 'type': 'forestry',
 'description': 'Generates organic foodstuffs',
 'populations will consume food before consuming natural resources': 'None',
 'planet_requirements': '{isHabitable: True}',
 'faction_augments': '{wealth: -3}',
 'renews_location_resource': '{organics: 10}',
 'owned_by': 'pop',
 'effort': '10',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'building',
 'id': '6058595786349'}

the `process_structure` function in the az-function app will look for the field `renews_location_resource`

In [22]:
resources_to_renew = yaml.safe_load(message['structure']['renews_location_resource'])
resources_to_renew

{'organics': 10}

In [37]:
popobjid = message['pop']['objid']
location_resources_query = f"g.V().has('objid','{popobjid}').out('inhabits').out('has').has('label','resource').valueMap()"
location_query = f"g.V().has('objid','{popobjid}').out('inhabits').valueMap()"
c.add_query(location_resources_query)
c.add_query(location_query)
c.run_queries()


2024-10-13 11:45:48,043 - DEBUG - Using selector: SelectSelector
2024-10-13 11:45:48,470 - DEBUG - Using selector: SelectSelector


In [50]:
location_resources = c.clean_nodes(c.res[location_resources_query])
location = c.clean_nodes(c.res[location_query])[0]
location

{'name': 'Leyeastpan',
 'class': 'terrestrial',
 'objid': '4162230164127',
 'pop_cap': 100,
 'atmosphere': '[{Argon: 0.185}, {Carbon Dioxide: 0.0}, {Helium: 0.037}, {Hydrogen: 0.231}, {Methane: 0.004}, {Nitrogen: 0.353}, {Oxygen: 0.104}, {Sodium: 0.087}]',
 'radius': 0.697,
 'mass': 0,
 'orbitsDistance': 0.682,
 'orbitsId': '7405736440710',
 'orbitsName': 'Nakly',
 'isSupportsLife': 'true',
 'isPopulated': 'true',
 'isHomeworld': 'true',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'planet',
 'id': '4162230164127'}

In [24]:
def augment_resources(c,resource,value):
    if resource['volume'] < resource['max_volume']:
        old_volume = resource['volume']
        new_volume = resource['volume'] + value
        print(f"EXOADMIN: resources {resource['name']}:{resource['objid']} changed by {value}, {old_volume}-> {new_volume}")
        renew_query = f"g.V().has('objid','{resource['objid']}').property('volume','{new_volume}')"
        print(renew_query)
        # c.run_query(renew_query)
    return resource

To create new resources we will need configurations

In [35]:
from app.functions import configurations
from app.functions import maths

In [34]:
resource_config = configurations.get_resource_configurations()
resource_config['resource']['resources']

{'organics': {'name': 'organics',
  'type': 'vegitable',
  'description': 'bilogical material that can be consumed by pops',
  'mean': 1000,
  'std': 100,
  'replenish_rate': 10},
 'minerals_common': {'name': 'common minerals',
  'type': 'mineral',
  'description': 'Iron and other common material used in constructuion of infrastructure',
  'mean': 100,
  'std': 10},
 'minerals_rare': {'name': 'rare minerals',
  'type': 'mineral',
  'description': 'lithium, silver and other rare minerals used in infrastructure and technology',
  'mean': 50,
  'std': 10},
 'water': {'name': 'water',
  'type': 'mineral',
  'description': 'H2O ready to be consumed, either frozen or in ice format',
  'mean': 10000,
  'std': 1000}}

In [48]:
def generate_new_resource(c,resource,location,value,resource_config):
    new_resource = resource_config['resource']['resources'][resource]
    new_resource['objid'] = maths.uuid()
    new_resource['volume'] = value
    data = {'nodes:': [new_resource], 'edges': [{'from': location['objid'], 'to': new_resource['objid'], 'label': 'has'}]}
    print(f"EXOADMIN: resources {new_resource['name']}:{new_resource['objid']} with volume: {value}")
    print(data)
    print(new_resource)
    
    return resource

In [26]:
for r in resources_to_renew.keys():
    # check if the resource is in the location
    resource_exists = len([i for i in location_resources if i['name']==r])>0
    if resource_exists:
        resource = [i for i in location_resources if i['name']==r][0]
        value = resources_to_renew[r]
        augment_resources(c,resource, value)
    if not resource_exists:
        print(f"EXOADMIN: resource {r} not found in location, and will be created")
        resource_config = configurations.get_resource_configurations()

    

In [53]:
resources_to_renew[r]

10

In [54]:
generate_new_resource(c,r,location,resources_to_renew[r],resource_config)

EXOADMIN: resources organics:0590888623964 created with volume: 10
{'nodes:': [{'name': 'organics', 'type': 'vegitable', 'description': 'bilogical material that can be consumed by pops', 'mean': 1000, 'std': 100, 'replenish_rate': 10, 'objid': '0590888623964', 'volume': 10}], 'edges': [{'from': '4162230164127', 'to': '0590888623964', 'label': 'has'}]}
{'name': 'organics', 'type': 'vegitable', 'description': 'bilogical material that can be consumed by pops', 'mean': 1000, 'std': 100, 'replenish_rate': 10, 'objid': '0590888623964', 'volume': 10}


'organics'